$$ \min f(x) \qquad \mbox{suj. a} \qquad \ell \leq x \leq u. $$

In [7]:
using Plots
gr()

Plots.GRBackend()

In [ ]:
f(x) = (x[1] - 1)^2 + 4*(x[2] - 1)^2
x = linspace(-0.5, 1.5, 100)
y = linspace(-0.5, 1.5, 100)
l = [-0.2 0.1]
u = [1.1 0.9]
contour(x, y, (x,y)->f([x;y]))
plot!([l[1], l[1], u[1], u[1], l[1]], [l[2], u[2], u[2], l[2], l[2]], c=:red, l=:dash)

Vamos considerar o problema com barreira

$$ \phi(x; \mu) = f(x) - \mu \log(x - l) - \mu \log(u - x). $$

In [ ]:
f(x) = (x[1] - 1)^2 + 4*(x[2] - 1)^2
axis = [-0.5, 1.5, -0.5, 1.5]

l = [-0.2; 0.1]
u = [1.1; 0.9]
x = linspace(l[1]+1e-4, u[1]-1e-4, 100)
y = linspace(l[2]+1e-4, u[2]-1e-4, 100)

ϕ(x, μ) = f(x) - μ*(sum(log(x-l)) + sum(log(u-x)))

μ = 1.0
contour(x, y, (x,y)->ϕ([x;y], μ))
plot!([l[1], l[1], u[1], u[1], l[1]], [l[2], u[2], u[2], l[2], l[2]], c=:red, l=:dash)
xlims!(axis[1], axis[2])
ylims!(axis[3], axis[4])

In [ ]:
using Interact

f(x) = (x[1] - 1)^2 + 4*(x[2] - 1)^2
axis = [-0.5, 1.5, -0.5, 1.5]

l = [-0.2; 0.1]
u = [1.1; 0.9]
x = linspace(l[1]+1e-4, u[1]-1e-4, 100)
y = linspace(l[2]+1e-4, u[2]-1e-4, 100)

ϕ(x, μ) = f(x) - μ*(sum(log(x-l)) + sum(log(u-x)))

@manipulate for μ = linspace(1.0, 10.0, 1000)
    contour(x, y, (x,y)->ϕ([x;y], log(μ)), levels=30)
    plot!([l[1], l[1], u[1], u[1], l[1]], [l[2], u[2], u[2], l[2], l[2]], c=:red, l=:dash)
    xlims!(axis[1], axis[2])
    ylims!(axis[3], axis[4])
end

As condições KKT desse problema são
$$ \nabla f(x) - \mu (X-L)^{-1}e + \mu (U-X)^{-1}e = 0, $$
onde $X$, $L$ e $U$ são matrizes diagonais com $x$, $\ell$ e $u$ na diagonal,
e $e$ é um vetor de uns. Note que, implicitamente $\ell < x < u$.

Agora, vamos definir vetores $z_L = \mu (X-L)^{-1}e$ e $z_U = \mu (U-X)^{-1}e$.
Desse modo, temos $(X-L)z_L = \mu e$ e $(U-X)z_U = \mu e$.
Ou ainda, $(X-L)Z_L e = \mu e$ e $(U-X)Z_U e = \mu e$.

Substituindo isso na condição KKT, temos
\begin{align*}
\nabla f(x) - z_L + z_U & = 0, \\
(X-L)Z_L e & = \mu e, \\
(U-X)Z_U e & = \mu e, \\
\end{align*}
De antes, temos $\ell < x < u$, e agora temos $z_L, z_U > 0$.
Essas condições, quando $\mu \to 0$, são as condições KKT do sistema original.

\begin{align*}
\nabla f(x) - z_L + z_U & = 0, \\
(X-L)Z_L e & = 0, \\
(U-X)Z_U e & = 0, \\
\end{align*}

Dessa maneira uma estratégia de resolução é usar um parâmetro $\sigma \in [0,1]$ para decidir o que seguir, usando as equações
\begin{align*}
\nabla f(x) - z_L + z_U & = 0, \\
(X-L)Z_L e & = \sigma\mu e, \\
(U-X)Z_U e & = \sigma\mu e, \\
\end{align*}

Novamente, vamos tentar aplicar Newton nesse sistema. Além disso, temos a preocupação dos sinais,
que serão tratados usando uma condição impedindo-os de atravessar a parede.

\begin{align*}
\left[
\begin{array}{ccc}
H(x) & -I & I \\
Z_L & X-L & 0 \\
-Z_U & 0 & U-X
\end{array}
\right]
\left[
\begin{array}{c}
d_x \\ d_L \\ d_U
\end{array}
\right]
=
-\left[
\begin{array}{c}
\nabla f(x) - z_L + z_U \\
(X-L)z_L - \mu e \\
(U-X)z_U - \mu e
\end{array}
\right]
\equiv
-\left[
\begin{array}{c}
r_x \\ r_L \\ r_U
\end{array}
\right]
\end{align*}

In [55]:
using ForwardDiff

f(x) = (x[1] - 1)^2 + 4*(x[2] - 1)^2
∇f(x) = ForwardDiff.gradient(f, x)
H(x) = ForwardDiff.hessian(f, x)

l = [-0.2; 0.1]
u = [1.1; 0.9]
x = [0.0; 0.5]
zL = 1.2ones(2)
zU = 1.3ones(2)
μ = 10.0

M = [H(x)  -eye(2)  eye(2); [diagm(zL); -diagm(zU)]  diagm([x-l; u-x])]

6x6 Array{Float64,2}:
  2.0   0.0  -1.0  -0.0  1.0  0.0
  0.0   8.0  -0.0  -1.0  0.0  1.0
  1.2   0.0   0.2   0.0  0.0  0.0
  0.0   1.2   0.0   0.4  0.0  0.0
 -1.3  -0.0   0.0   0.0  1.1  0.0
 -0.0  -1.3   0.0   0.0  0.0  0.4

In [56]:
rx = ∇f(x) - zL + zU
rL = (x-l).*zL - μ
rU = (u-x).*zU - μ
d = -M\[rx; rL; rU]

6-element Array{Float64,1}:
  4.67327 
  0.280702
 20.7604  
 22.9579  
 13.3139  
 24.6123  

In [57]:
dx, dL, dU = d[1:2], d[3:4], d[5:6]

([4.673267326732671,0.280701754385964],[20.760396039603954,22.9578947368421],[13.313861386138612,24.612280701754386])

In [58]:
l .< x + dx .< u

2-element BitArray{1}:
 false
  true

Quanto até a barreira? O maior $\alpha > 0$ tal que
$$ \ell \leq x + \alpha d \leq u. $$
Mas na prática, não podemos ir pra barreira, então colocar um valor pequeno para dentro
da barreira, (i.e., não pode ser 0.0, mas pode ser 1e-4).
Por fim, pra deixar mais interessante, o valor tem que ser relativo à quanto falta para chegar lá,
uma vez que um passo "cego" poderia resultar num $\alpha$ negativo ou zero.
Isso resulta na condição abaixo, chamada **fração-para-a-fronteira**:

$$ \ell + (1-\tau)(x-l) \leq x + \alpha d \leq u - (1-\tau)(u-x), $$

ou seja,

$$ -\tau(x-l) \leq \alpha d \leq \tau (u-x), $$

para algum $\tau \in (0,1)$. Quanto mais perto de $1$, maior o passo possível.

Daí,
Se $d_i > 0$, então $\alpha \leq \tau \dfrac{u_i - x_i}{d_i}.$
Se $d_i < 0$, então $\alpha \leq \tau \dfrac{x_i - \ell_i}{d_i}.$
Por outro lado, gostaria da andar todo o passo de Newton, então $\alpha = 1$ é o ideal.

In [59]:
τ = 0.9
α = 1.0
for i = 1:2
    if dx[i] > 0
        α = min(α, τ*(u[i] - x[i])/d[i])
    elseif dx[i] < 0
        α = min(α, -τ*(x[i] - l[i])/d[i])
    end
    if dL[i] < 0
        α = min(α, -τ*zL[i]/dL[i])
    end
    if dU[i] < 0
        α = min(α, -τ*zU[i]/dU[i])
    end
end
α

0.21184322033898317

In [60]:
x + α*dx, zL + α*dL, zU + α*dU

([0.9900000000000001,0.5594647636039249],[5.597949152542374,6.063474353256023],[4.120451271186442,6.513944803746658])

In [91]:
using ForwardDiff

function ponto_interior_caixa(f, x, l, u; tol = 1e-5, maxiter = 1000, maxtime = 60, τ = 0.9)
    exit_flag = 0
    ∇f(x) = ForwardDiff.gradient(f, x)
    H(x) = ForwardDiff.hessian(f, x)
    
    iter = 0
    start_time = time()
    elapsed_time = 0.0
    
    n = length(x)
    x = [l[i] < x[i] < u[i] ? x[i] : (l[i]+u[i])/2 for i = 1:n]
    fx = f(x)
    ∇fx = ∇f(x)
    Hx = H(x)
    zL = ones(n)
    zU = ones(n)
    
    μ = (dot(x-l,zL) + dot(u-x,zU))/2n    
    σ = 1.0
    
    rx = ∇fx - zL + zU
    rL = (x - l).*zL - μ*σ
    rU = (u - x).*zU - μ*σ
    
    
    while norm([rx;rL;rU]) > tol || μ > tol
        # Idealmente isso deveria ser esparso
        M = [Hx  -eye(n)  eye(n); [diagm(zL); -diagm(zU)]  diagm([x-l; u-x])]
        d = -M\[rx; rL; rU]
        dx, dL, dU = d[1:n], d[n+1:2n], d[2n+1:3n]
        # Tamanho do passo

        α = 1.0
        for i = 1:n
            if dx[i] > 0
                α = min(α, τ*(u[i] - x[i])/dx[i])
            elseif dx[i] < 0
                α = min(α, -τ*(x[i] - l[i])/dx[i])
            end
            if dL[i] < 0
                α = min(α, -τ*zL[i]/dL[i])
            end
            if dU[i] < 0
                α = min(α, -τ*zU[i]/dU[i])
            end
        end
        
        x = x + α*dx
        zL = zL + α*dL
        zU = zU + α*dU
        μ = (dot(x-l, zL) + dot(u-x, zU))/2n
        σ *= 0.9
        
        fx = f(x)
        ∇fx = ∇f(x)
        Hx = H(x)
        
        rx = ∇fx - zL + zU
        rL = (x - l).*zL - μ*σ
        rU = (u - x).*zU - μ*σ
        
        iter = iter + 1
        if iter >= maxiter
            exit_flag = 1
            break
        end
        elapsed_time = time() - start_time
        if elapsed_time >= maxtime
            exit_flag = 2
            break
        end
    end
    return x, fx, rx, exit_flag, iter, elapsed_time # Precisamos retornar o ponto encontrado
end

ponto_interior_caixa (generic function with 1 method)

In [92]:
f(x) = (x[1] - 1)^2 + 4*(x[2] - 1)^2
l = [-0.2; 0.1]
u = [1.1; 0.9]
x = [0.0; 0.5]
ponto_interior_caixa(f, x, l, u, maxiter=1000)

([0.9999924176556605,0.8999979338230314],0.04000165301614318,[5.288196096298048e-17,3.3306690738754696e-16],0,16,0.015563011169433594)

In [93]:
xx = linspace(-0.5, 1.5, 100)
yy = linspace(-0.5, 1.5, 100)
l = [-0.2; 0.1]
u = [1.1; 0.9]
contour(xx, yy, (x,y)->f([x;y]))
plot!([l[1], l[1], u[1], u[1], l[1]], [l[2], u[2], u[2], l[2], l[2]], c=:red, l=:dash)

x = [0.0; 0.5]
scatter!([x[1]], [x[2]], c=:blue, leg=false)
xn = copy(x)
for i = 1:100
    xp = copy(xn)
    xn, _ = ponto_interior_caixa(f, x, l, u, maxiter=i)
    plot!([xp[1], xn[1]], [xp[2], xn[2]], c=:blue)
    scatter!([xn[1]], [xn[2]], c=:blue, ms=2)
end
plot!()

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 0 
 
 
 1 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#fb9c07; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 49.6018,331.686 54.3725,332.698 59.1431,333.697 63.1073,334.515 63.9137,334.679 68.6844,335.636 73.455,336.579 78.2256,337.509 81.1564,338.071 82.9963,338.419 
 87.7669,339.308 92.5375,340.184 97.3081,341.046 100.572,341.627 102.079,341.891 106.849,342.714 111.62,343.525 116.391,344.322 121.161,345.106 121.638,345.183 
 125.932,345.866 130.703,346.613 135.473,347.347 140.244,348.068 144.764,348.739 145.014,348.775 149.785,349.46 154.556,350.132 159.326,350.792 164.097,351.439 
 168.868,352.073 170.568,352.294 173.638,352.689 178.409,353.288 183.179,353.876 187.95,354.451 192.721,355.013 197.491,355.563 200.044,355.85 202.262,356.096 
 207.033,356.614 211.803,357.118 216.574,357.611 221.345,358.091 226.115,358.559 230.886,359.014 235.105,359.406 235.656,359.456 240.427,359.881 245.198,360.293 
 249.968,360.693 254.739,361.081 259.51,361.457 264.28,361.821 269.051,362.172 273.821,362.511 278.592,362.838 280.467,362.962 283.363,363.15 288.133,363.448 
 292.904,363.734 297.675,364.009 302.445,364.271 307.216,364.521 311.986,364.759 316.757,364.985 321.528,365.199 326.298,365.401 331.069,365.591 335.84,365.769 
 340.61,365.934 345.381,366.088 350.152,366.23 354.922,366.359 359.693,366.477 361.531,366.517 
 "/>
 
<polyline clip-path="url(#clip02)" style="stroke:#f57d15; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 521.894,350.326 517.124,350.635 512.353,350.932 507.582,351.215 502.812,351.486 498.041,351.744 493.27,351.99 488.5,352.222 486.936,352.294 483.729,352.44 
 478.959,352.644 474.188,352.836 469.417,353.015 464.647,353.181 459.876,353.335 455.105,353.477 450.335,353.605 445.564,353.722 440.794,353.825 436.023,353.916 
 431.252,353.995 426.482,354.061 421.711,354.114 416.94,354.155 412.17,354.183 407.399,354.199 402.628,354.202 397.858,354.193 393.087,354.171 388.317,354.136 
 383.546,354.089 378.775,354.029 374.005,353.957 369.234,353.872 364.463,353.775 359.693,353.665 354.922,353.542 350.152,353.407 345.381,353.26 340.61,353.1 
 335.84,352.927 331.069,352.742 326.298,352.544 321.528,352.333 320.696,352.294 316.757,352.108 311.986,351.869 307.216,351.617 302.445,351.352 297.675,351.075 
 292.904,350.785 288.133,350.483 283.363,350.167 278.592,349.839 273.821,349.498 269.051,349.144 264.28,348.778 263.784,348.739 259.51,348.394 254.739,347.996 
 249.968,347.586 245.198,347.162 240.427,346.726 235.656,346.277 230.886,345.814 226.115,345.339 224.586,345.183 221.345,344.846 216.574,344.338 211.803,343.816 
 207.033,343.281 202.262,342.734 197.491,342.173 192.956,341.627 192.721,341.598 187.95,341.002 183.179,340.393 178.409,339.77 173.638,339.135 168.868,338.485 
 165.886,338.071 164.097,337.819 159.326,337.133 154.556,336.433 149.785,335.72 145.014,334.993 141.935,334.515 140.244,334.249 135.473,333.484 130.703,332.705 
 125.932,331.912 121.161,331.105 120.315,330.96 116.391,330.274 111.62,329.427 106.849,328.566 102.079,327.691 100.537,327.404 97.3081,326.793 92.5375,325.875 
 87.7669,324.944 82.9963,323.998 82.2501,323.848 78.2256,323.025 73.455,322.036 68.6844,321.032 65.2221,320.292 63.9137,320.009 59.1431,318.959 54.3725,317.894 
 49.6018,316.815 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#e65e2d; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 49.6018,300.821 54.3725,301.997 56.4995,302.514 59.1431,303.145 63.9137,304.269 68.6844,305.377 71.7065,306.069 73.455,306.463 78.2256,307.521 82.9963,308.564 
 87.7669,309.592 87.9255,309.625 92.5375,310.587 97.3081,311.567 102.079,312.531 105.343,313.181 106.849,313.476 111.62,314.394 116.391,315.298 121.161,316.187 
 124.161,316.737 125.932,317.056 130.703,317.901 135.473,318.731 140.244,319.547 144.682,320.292 145.014,320.347 149.785,321.121 154.556,321.881 159.326,322.626 
 164.097,323.356 167

In [94]:
f(x) = (x[1] - 1)^2 + 100*(x[2] - x[1]^2)^2
l = [-0.2; 0.1]
u = [1.1; 0.9]
x = [0.0; 0.5]
ponto_interior_caixa(f, x, l, u, maxiter=1000)

([-0.19999927917355065,0.10000014416333115],1.8000034599564416,[-2.473961759883703e-9,-1.546263835625644e-9],0,16,0.05794215202331543)

In [98]:
xx = linspace(-0.5, 1.5, 100)
yy = linspace(-0.5, 1.5, 100)
contour(xx, yy, (x,y)->f([x;y]))
plot!([l[1], l[1], u[1], u[1], l[1]], [l[2], u[2], u[2], l[2], l[2]], c=:red, l=:dash)

x = [0.4; 0.5]
scatter!([x[1]], [x[2]], c=:blue, leg=false)
xn = copy(x)
for i = 1:100
    xp = copy(xn)
    xn, _ = ponto_interior_caixa(f, x, l, u, maxiter=i)
    plot!([xp[1], xn[1]], [xp[2], xn[2]], c=:blue)
    scatter!([xn[1]], [xn[2]], c=:blue, ms=2)
end
plot!()

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 0 
 
 
 1 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#0d0829; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 236.013,366.517 240.427,364.582 243.931,362.962 245.198,362.368 249.968,360.002 251.106,359.406 254.739,357.484 257.686,355.85 259.51,354.831 263.846,352.294 
 264.28,352.038 269.051,349.091 269.596,348.739 273.821,345.995 275.021,345.183 278.592,342.757 280.188,341.627 283.363,339.376 285.128,338.071 288.133,335.85 
 289.867,334.515 292.904,332.18 294.428,330.96 297.675,328.367 298.832,327.404 302.445,324.411 303.097,323.848 307.216,320.313 307.239,320.292 311.218,316.737 
 311.986,316.054 315.093,313.181 316.757,311.654 318.879,309.625 321.528,307.117 322.588,306.069 326.227,302.514 326.298,302.444 329.72,298.958 331.069,297.601 
 333.163,295.402 335.84,292.63 336.565,291.846 339.878,288.291 340.61,287.511 343.105,284.735 345.381,282.245 346.312,281.179 349.452,277.623 350.152,276.838 
 352.508,274.068 354.922,271.284 355.562,270.512 358.529,266.956 359.693,265.581 361.453,263.4 364.385,259.845 364.463,259.75 367.188,256.289 369.234,253.751 
 370.016,252.733 372.774,249.177 374.005,247.615 375.499,245.622 378.217,242.066 378.775,241.341 380.849,238.51 383.526,234.954 383.546,234.928 386.072,231.399 
 388.317,228.352 388.673,227.843 391.174,224.287 393.087,221.635 393.706,220.731 396.161,217.176 397.858,214.775 398.632,213.62 401.041,210.064 402.628,207.772 
 403.458,206.508 405.818,202.953 407.399,200.624 408.188,199.397 410.499,195.841 412.17,193.333 412.829,192.285 415.09,188.73 416.94,185.898 417.388,185.174 
 419.596,181.618 421.711,178.322 421.868,178.062 424.024,174.507 426.256,170.951 426.482,170.593 428.379,167.395 430.555,163.839 431.252,162.713 432.667,160.284 
 434.786,156.728 436.023,154.694 436.893,153.172 438.955,149.616 440.794,146.54 441.063,146.061 443.067,142.505 445.142,138.949 445.564,138.231 447.128,135.393 
 449.142,131.837 450.335,129.774 451.143,128.282 453.096,124.726 455.105,121.192 455.117,121.17 457.01,117.614 458.967,114.059 459.876,112.435 460.887,110.503 
 462.782,106.947 464.647,103.561 464.734,103.391 466.566,99.8357 468.461,96.2799 469.417,94.5175 470.325,92.7242 472.155,89.1684 474.049,85.6127 474.188,85.3525 
 475.828,82.0569 477.654,78.5012 478.959,76.0226 479.487,74.9454 481.246,71.3897 483.066,67.8339 483.729,66.5555 484.827,64.2782 486.578,60.7224 488.392,57.1666 
 488.5,56.9553 490.087,53.6109 491.828,50.0551 493.27,47.1933 493.595,46.4994 495.267,42.9436 496.997,39.3879 498.041,37.2875 498.711,35.8321 500.37,32.2764 
 502.087,28.7206 502.812,27.2419 503.754,25.1649 505.397,21.6091 507.099,18.0534 507.582,17.0539 508.725,14.4976 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#270b52; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 521.894,50.9504 520.673,53.6109 519.067,57.1666 517.494,60.7224 517.124,61.5643 515.861,64.2782 514.234,67.8339 512.641,71.3897 512.353,72.0347 510.981,74.9454 
 509.334,78.5012 507.721,82.0569 507.582,82.3621 506.029,85.6127 504.363,89.1684 502.812,92.5436 502.724,92.7242 501.003,96.2799 499.318,99.8357 498.041,102.575 
 497.64,103.391 495.901,106.947 494.199,110.503 493.27,112.464 492.475,114.059 490.72,117.614 489.001,121.17 488.5,122.212 487.228,124.726 485.457,128.282 
 483.729,131.823 483.722,131.837 481.897,135.393 480.11,138.949 478.959,141.273 478.316,142.505 476.478,146.061 474.677,149.616 474.188,150.587 472.821,153.172 
 470.969,156.728 469.417,159.76 469.136,160.284 467.232,163.839 465.367,167.395 464.647,168.78 463.462,170.951 461.547,174.507 459.876,177.666 459.656,178.062 
 457.689,181.618 455.764,185.174 455.105,186.398 453.791,188.73 451.815,192.285 450.335,194.991 449.848,195.841 447.821,199.397 445.836,202.953 445.564,203.441 
 443.777,206.508 441.742,210.064 440.794,211.739 439.678,213.62 437.594,217.176 436.023,219.9 435.521,220.731 433.387,224.287 431.298,227.843 431.